## Ejemplo 1: API del Banco Mundial (World Bank)

La API de indicadores del Banco Mundial provee acceso programático a cerca de 16,000 series de tiempo de indicadores disponibles en plataformas online como Databank (https://databank.worldbank.org/). Muchas series de tiempo llegan hasta más de 50 años atrás, y pueden ser usadas para crear aplicaciones y análisis interesantes.

La información general de la API del Banco Mundial está diponible en este link:

https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-the-indicators-api-documentation

Y en particular, las isntrucciones para consultar la API están acá:

https://datahelpdesk.worldbank.org/knowledgebase/articles/898581


Primero veamos como extrar el listado de todos los datos de la API Country, en formato JSON.

In [1]:
import requests
import pandas as pd

url='http://api.worldbank.org/v2/country'

In [ ]:
# Primero veamos el caso de una query general, sin definir parámetros.
response=requests.get(url)
response.content

Por defecto, la API entrega resultados en formato XML. Para obtenerlos en formato JSON, es necesario especificar
el parámetro `format`

![image.png](attachment:image.png)

In [ ]:
#Por defecto, la API entrega resultados en formato XML. Para obtenerlos en formato JSON, es necesario especificar
#el parámetro 'format'

response=requests.get(url,params={'format':'json'})
response.content

In [ ]:
#Otra opción: incluir parámetros directamente en el string url
url_query='http://api.worldbank.org/v2/country?format=json'
response=requests.get(url_query)
response.content

Inspeccionemos ahora el contenido de la respuesta:

In [ ]:
json=response.json()
json

In [6]:
json[0]

{'page': 1, 'pages': 6, 'per_page': '50', 'total': 299}

La respuesta de la API es una lista de diccionarios. El primero de ellos contiene información general de los datos, y el segundo contiene los datos de interés relativos a países.

In [ ]:
json[1]

In [ ]:
#Podemos llevar estos datos a un DataFrame de pandas
df=pd.DataFrame(json[1])
df

Dado que los datos contienen diccionarios anidados, es necesario usar la función `json_normalize`para generar un DataFrame aplanado.

In [ ]:
df=pd.json_normalize(json[1])
df

Podemos repetir el ejercicio completo para una consulta más específica, por ejemplo, solicitando sólo la información de los países de bajo nivel de ingresos (Low Income Country). De acuerdo a la documentación de la API, esto requiere definir el parámetro `incomeLevel=LIC`

In [ ]:
response=requests.get(url,params={'format':'json','incomeLevel':'LIC'})
json=response.json()
df2=pd.json_normalize(json[1])
df2.head(10)

## Ejemplo 2: API Yelp Fusion ¿Dónde puedo comer en Santiago?

La aplicación Yelp permite a los usuarios calificar y enviar comentarios sobre distintos negocios, y disponibiliza esta información mediante un conjunto de APIs.

https://www.yelp.com/developers/documentation/v3/get_started

Para conectarse a estas APIs, se requiere una clave privada de autentificación (gratuita), que puede ser creada siguiendo las instrucciones en:

https://www.yelp.com/developers/documentation/v3/authentication

En esje ejmplo, usaremos la API *Businesses Search* para obtenre un listado de restaurants en la ciudad de Santiago. La información requerida para hacer la búsqueda está disponible en:

https://www.yelp.com/developers/documentation/v3/business_search

Una característica importante de la API, es que entrega un **máximo de 1000 resultados**. Además, cada query tiene un límite de 50 resultados. Por lo tanto, es necesario **iterar y realizar varias queries para obtener el máximo de 1000 registros**. Para esto, se utiliza el parámetro `offset`, que permite ir avanzando en la lista.

In [12]:
#url de la API
api_url='https://api.yelp.com/v3/businesses/search'

#estos datos corresponden a una cuenta de usuario creada previamente
clientid='GWOCZh9-BmZxtdsAjr7Gug'
apikey='FHVvXoNmTXIl9DuxYis7AV5uLPujm9MLwrhgs5NgvCfaOxd3V6mxt6dQU8eEqYJiGxe816XATx7ufWjbMWqbV-2Uku1jxBJv8BGRC74NroLPl27PDQqs0tDixit-YHYx'
headers={'Authorization':'Bearer %s'%apikey}

In [13]:
params={'term':'restaurants','location':'RM Santiago, Chile','limit':50}
response=requests.get(api_url,params=params,headers=headers)
data=response.json()
data

{'businesses': [{'id': '8N6Y3HsLXHBeFwhlO0YxwQ',
   'alias': 'barrica-94-santiago',
   'name': 'Barrica 94',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/FRYceqc7DdBnvQvDj7C2ZA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/barrica-94-santiago?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
   'review_count': 76,
   'categories': [{'alias': 'wine_bars', 'title': 'Wine Bars'},
    {'alias': 'chilean', 'title': 'Chilean'},
    {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': -33.4342880070125,
    'longitude': -70.6352240592241},
   'transactions': [],
   'price': '$$',
   'location': {'address1': 'Bellavista 052',
    'address2': '',
    'address3': '',
    'city': 'Santiago',
    'zip_code': '8320000',
    'country': 'CL',
    'state': 'RM',
    'display_address': ['Bellavista 052', 'RM 8320000 Santiago', 'Chile']},


In [ ]:
#Veamos los keys del diccionario recibido
data.keys()

dict_keys(['businesses', 'total', 'region'])

In [14]:
#El primer elemento del diccionario indica el total de restaurants existentes en la API
print('En total la base de datos registra %d restaurants'%data['total'])

En total la base de datos registra 2300 restaurants


In [ ]:
data['businesses']

La data entregada por la API Yelp API es un objeto en formato JSON anidado, es decir, un diccionario donde algunos de los valores de atributos corresponden a su vez a listas o diccionarios.

Como vimos anteriormente, podemos llevar estos datos a un formato "aplanado" o "flattened",utilizando la función `json_normalize()` :

https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html

In [ ]:
#normalizamos el contenido del diccionario "businesses"
rests = pd.json_normalize(data["businesses"],sep='_',record_path=['categories'],meta=['name','price','rating','review_count','distance',['coordinates','latitude'],['coordinates','longitude'],['location','address1']],errors='ignore')
rests.head()

La API cuenta con información relativa a 2300 restaurantes, pero en cada consulta se extrae una página con 50 resultados. Por lo tanto, para extraer todos los datos, será necesario iterar el proceso. El parámetro `offset` de la API indica el número de registros a avanzar en cada consulta (avanzamos de 50 en 50).

In [ ]:
#iteramos
offset=0

#creamos una lista para reunir los dataframes creados para cada una de las consultas
allrests=[]

#iteramos hasta llegar a 1000 resultados (el máximo que permite la API)
while offset<=950:
    
    #parámetros de la consulta a la API
    params={'term':'restaurants','location':'RM Santiago, Chile','limit':50,'offset':offset}

    response=requests.get(api_url,params=params,headers=headers)
    data=response.json()
    
    #dataframe con 50 resultados
    rests = pd.json_normalize(data["businesses"],sep='_',record_path=['categories'],meta=['name','price','rating','review_count','distance',['coordinates','latitude'],['coordinates','longitude'],['location','address1']],errors='ignore')

    #agregamos este resultado a la lista de dataframes
    allrests.append(rests)
    
    #avanzamos 50 registros
    offset=offset+50
    

In [ ]:
#Finalmente concatenamos todos los dataframes para crear uno solo con todos los resultados
rests=pd.concat(allrests,ignore_index=True)
rests

,alias,title,name,price,rating,review_count,distance,coordinates_latitude,coordinates_longitude,location_address1
0,wine_bars,Wine Bars,Barrica 94,$$,4.5,76,1395.735019,-33.434288,-70.635224,Bellavista 052
1,chilean,Chilean,Barrica 94,$$,4.5,76,1395.735019,-33.434288,-70.635224,Bellavista 052
2,cocktailbars,Cocktail Bars,Barrica 94,$$,4.5,76,1395.735019,-33.434288,-70.635224,Bellavista 052
3,seafood,Seafood,Aquí Está Coco,$$$$,4.5,57,2366.370671,-33.424063,-70.616624,La Concepción 236
4,latin,Latin American,Restaurant Peumayén,$$$$,4.5,68,1504.86354,-33.432807,-70.634821,Constitución 136
...,...,...,...,...,...,...,...,...,...,...
1448,bistros,Bistros,Mossto Brewfood,$$,3.0,4,429.029964,-33.447819,-70.625375,Condell 1460
1449,burgers,Burgers,Burgs,NaN,4.0,1,2933.309968,-33.42056,-70.6108,Nueva de Lyon 105
1450,hotdog,Hot Dogs,Carlito,NaN,3.5,2,2862.989554,-33.469612,-70.622887,San Eugenio 1900
1451,hotdogs,Fast Food,Carlito,NaN,3.5,2,2862.989554,-33.469612,-70.622887,San Eugenio 1900
